In [3]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from xgboost import XGBRegressor

from sklearn.pipeline import make_pipeline, make_union
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.utils import check_array

from sklearn.metrics import r2_score

In [4]:
# pandas
import pandas as pd
from pandas import Series,DataFrame
from sklearn.preprocessing import LabelEncoder
#gb est un objet de type gradient boosting
#aucun paramètre n’est spécifié, l’objet utilise les paramètres par défaut
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingRegressor()
print(gb)
#afficher l’objet revient à afficher ses paramètres

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


Nous remarquons entres autres, en nous référant à la documentation
        la fonction de coût déviance est utilisée (loss = ‘deviance’)      
        100 arbres sont élaborés (n_estimators = 100)
       
        la constante d’apprentissage (learning rate) = 0.1
        
        la profondeur maximal des arbres individuels est fixé à max_depth = 3, la valeur est assez faible et nous prémunit du surapprentissage
       
        les effectifs sur les nœuds, avant et après segmentation, ne sont pas opérantes avec des valeurs aussi faibles (min_samples_split = 2, min_samples_leaf  = 1)
        
        il n’y a pas de tirage aléatoire des individus (pas de ‘’stochastic gradient boosting’’, 
        [GBM, page 16]) pour l’élaboration des arbres intermédiaires (subsample = 1.0)
       
        il n’y a pas de tirage aléatoire de variables à  la  Random  Forest lors  des  segmentations dans les arbres individuels (max_features = None).

In [5]:
#récuperation des données 
#données pour le train
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [6]:
#transfomer les objets en code numérique 

for c in data_train:
    if data_train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(data_train[c].values)+list(data_test[c].values))
        data_train[c] = lbl.transform(list(data_train[c].values))
        data_test[c]= lbl.transform(list(data_test[c].values))
     


In [7]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to X385
dtypes: float64(1), int64(377)
memory usage: 12.1 MB


In [8]:
from sklearn.cross_validation import train_test_split

y_train= data_train["y"].values
X_train= data_train.drop(["y","ID"], axis=1)
X_test = data_test.drop(["ID"], axis=1)

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 376 entries, X0 to X385
dtypes: int64(376)
memory usage: 12.1 MB


In [10]:
gb.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [11]:
#prédiction sur l’échantillon test
y_pred = gb.predict(X_test)
#évaluation : calcul du taux d’erreur
#taux d’erreur = 1 -taux de succès
#from sklearn import metrics
#err = 1.0 -metrics.accuracy_score(y_train,y_pred)
print(y_pred)

[  97.36299648  111.33338688   97.01159779 ...,   91.42375117  109.25050119
   91.20497367]


In [12]:
a=y_pred-y_train

In [13]:
y_pred.mean()

101.00379285028257

In [14]:
y_train.mean()

100.66931812782134

In [15]:

print('score R2 pour le model 3')
print(r2_score(y_train,y_pred))

score R2 pour le model 3
-0.569704170041


In [16]:
id_test = data_test['ID'].values
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('test_GB.csv', index=False)

In [17]:
#classe pour la grille de recherche des paramètres en validation croisée
#par défaut 3-fold validation croisée
from sklearn.grid_search import GridSearchCV
#combinaisons de paramètres à tester
parametres = {"learning_rate":[0.3,0.2,0.1,0.05,0.01],"max_depth":[2,3,4,5,6],"subsample":[1.0,0.8,0.5],"max_features":[None,'sqrt','log2']}
#méthode d’apprentissage à tester
gbr = GradientBoostingRegressor()
#object (instance de classe) pour la grille de recherche
gril = GridSearchCV(estimator=gbr,param_grid=parametres,scoring="accuracy") 
#lancer la recherche sur les données d’apprentissage
resultats = gril.fit(X_train,y_train)
#affichage des performances (scores)
print(resultats.grid_scores_)
#meilleures performances (scores)
print(resultats.best_score_)
#meilleurs paramètres au sens de la validation croisée

C:\anacanda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


ValueError: continuous is not supported